In [4]:
pip install kagglehub


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score
import kagglehub

# Download dataset from Kaggle
path = kagglehub.dataset_download("zalando-research/fashionmnist")
print("Path to dataset files:", path)

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load FashionMNIST dataset
train_dataset = datasets.FashionMNIST(root=path, train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root=path, train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define CNN model
class CNNModel(nn.Module):
    def __init__(self, kernel_size, pool_type):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=kernel_size, padding=1)
        self.pool = nn.MaxPool2d(2, 2) if pool_type == 'max' else nn.AvgPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=1)
        self.fc1 = nn.Linear(64 * (28 // (2 ** 2)) * (28 // (2 ** 2)), 128)
        self.fc2 = nn.Linear(128, 10)  # FashionMNIST has 10 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * (28 // (2 ** 2)) * (28 // (2 ** 2)))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Training function
def train_model(model, train_loader, criterion, optimizer, scheduler, writer, epochs, early_stopping_patience):
    model.train()
    best_loss = float('inf')
    early_stopping_counter = 0

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        scheduler.step()
        avg_loss = running_loss / len(train_loader)
        writer.add_scalar('Loss/train', avg_loss, epoch)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}')

        if avg_loss < best_loss:
            best_loss = avg_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= early_stopping_patience:
                print("Early stopping triggered.")
                break

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

# Main function
def main():
    kernel_sizes = [3, 5, 7]
    pool_types = ['max', 'avg']
    epochs_list = [5, 50, 100, 250, 350]
    optimizers = ['SGD', 'RMSprop', 'Adam']

    for kernel_size in kernel_sizes:
        for pool_type in pool_types:
            for epochs in epochs_list:
                for optimizer_name in optimizers:
                    model = CNNModel(kernel_size, pool_type)
                    criterion = nn.CrossEntropyLoss()
                    if optimizer_name == 'SGD':
                        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
                    elif optimizer_name == 'RMSprop':
                        optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.99)
                    elif optimizer_name == 'Adam':
                        optimizer = optim.Adam(model.parameters(), lr=0.001)

                    scheduler = StepLR(optimizer, step_size=20, gamma=0.1)
                    writer = SummaryWriter(f'runs/kernel_{kernel_size}_pool_{pool_type}_epochs_{epochs}_optimizer_{optimizer_name}')

                    print(f'Training with kernel_size={kernel_size}, pool_type={pool_type}, epochs={epochs}, optimizer={optimizer_name}')
                    train_model(model, train_loader, criterion, optimizer, scheduler, writer, epochs, early_stopping_patience=10)
                    accuracy = evaluate_model(model, test_loader)
                    print(f'Accuracy: {accuracy:.4f}')
                    writer.add_scalar('Accuracy/test', accuracy, epochs)
                    writer.close()

if __name__ == '__main__':
    import torch.nn.functional as F  # Impor F di sini
    main()


Path to dataset files: /root/.cache/kagglehub/datasets/zalando-research/fashionmnist/versions/4
Training with kernel_size=3, pool_type=max, epochs=5, optimizer=SGD
Epoch 1/5, Loss: 0.4580
Epoch 2/5, Loss: 0.2820
Epoch 3/5, Loss: 0.2381
Epoch 4/5, Loss: 0.2085
Epoch 5/5, Loss: 0.1850
Accuracy: 0.9131
Training with kernel_size=3, pool_type=max, epochs=5, optimizer=RMSprop
Epoch 1/5, Loss: 0.4193
Epoch 2/5, Loss: 0.2647
Epoch 3/5, Loss: 0.2186
Epoch 4/5, Loss: 0.1875
Epoch 5/5, Loss: 0.1622
Accuracy: 0.9155
Training with kernel_size=3, pool_type=max, epochs=5, optimizer=Adam
Epoch 1/5, Loss: 0.3983
Epoch 2/5, Loss: 0.2541
Epoch 3/5, Loss: 0.2093
Epoch 4/5, Loss: 0.1744
Epoch 5/5, Loss: 0.1494
Accuracy: 0.9165
Training with kernel_size=3, pool_type=max, epochs=50, optimizer=SGD
Epoch 1/50, Loss: 0.4658


KeyboardInterrupt: 